In [1]:
import pandas as pd
import duckdb
import chardet

In [2]:
data = pd.read_excel("idea_export_voter_turnout_database_region.xlsx")
data.head()

,Country,ISO2,ISO3,Election Type,Year,Voter Turnout,Total vote,Registration,VAP Turnout,Voting age population,Population,Invalid votes,Compulsory voting
0,Afghanistan,AF,AFG,Parliamentary,2018-01-01,45.23%,"4,000,000","8,843,151",21.81%,"18,340,292","34,940,837",NaN,No
1,Afghanistan,AF,AFG,Parliamentary,2010-01-01,35.14%,"4,216,594","12,000,000",29.71%,"14,191,908","29,120,727",4.42%,No
2,Afghanistan,AF,AFG,Parliamentary,2005-01-01,49.37%,"6,406,615","12,977,336",51.71%,"12,389,532","26,334,702",5.10%,No
3,Afghanistan,AF,AFG,Presidential,2019-01-01,19.00%,"1,824,401","9,600,000",9.63%,"18,938,369","35,780,458",NaN,No
4,Afghanistan,AF,AFG,Presidential,2014-06-14,38.90%,"8,109,493","20,845,988",50.03%,"16,208,255","31,822,848",1.68%,No


In [3]:
data.memory_usage()

Index                      132
Country                  29192
ISO2                     29192
ISO3                     29192
Election Type            29192
Year                     29192
Voter Turnout            29192
Total vote               29192
Registration             29192
VAP Turnout              29192
Voting age population    29192
Population               29192
Invalid votes            29192
Compulsory voting        29192
dtype: int64

In [4]:
data.shape

(3649, 13)

In [5]:
len(list(data["Country"].unique()))

203

In [6]:
data.dtypes

Country                  object
ISO2                     object
ISO3                     object
Election Type            object
Year                     object
Voter Turnout            object
Total vote               object
Registration             object
VAP Turnout              object
Voting age population    object
Population               object
Invalid votes            object
Compulsory voting        object
dtype: object

In [7]:
numeric_cols = ["Total vote", "Registration", "Voting age population", "Population", "Voter Turnout", "VAP Turnout", "Invalid votes"]
# Remove the commas and % in the columns and convert to floats
for col in numeric_cols:
    data[col] = data[col].str.replace(",", "").str.replace("%", "")
    data[col] = data[col].astype("float32")
data.dtypes

Country                   object
ISO2                      object
ISO3                      object
Election Type             object
Year                      object
Voter Turnout            float32
Total vote               float32
Registration             float32
VAP Turnout              float32
Voting age population    float32
Population               float32
Invalid votes            float32
Compulsory voting         object
dtype: object

In [8]:
# change year col to datetime
data["Year"] = pd.to_datetime(data["Year"])

In [9]:
data[["Election Type", "Compulsory voting"]] = data[["Election Type", "Compulsory voting"]].astype("category")

In [10]:
data.dtypes

Country                          object
ISO2                             object
ISO3                             object
Election Type                  category
Year                     datetime64[ns]
Voter Turnout                   float32
Total vote                      float32
Registration                    float32
VAP Turnout                     float32
Voting age population           float32
Population                      float32
Invalid votes                   float32
Compulsory voting              category
dtype: object

In [11]:
data.memory_usage()

Index                      132
Country                  29192
ISO2                     29192
ISO3                     29192
Election Type             3781
Year                     29192
Voter Turnout            14596
Total vote               14596
Registration             14596
VAP Turnout              14596
Voting age population    14596
Population               14596
Invalid votes            14596
Compulsory voting         3773
dtype: int64

In [12]:
data.head()

,Country,ISO2,ISO3,Election Type,Year,Voter Turnout,Total vote,Registration,VAP Turnout,Voting age population,Population,Invalid votes,Compulsory voting
0,Afghanistan,AF,AFG,Parliamentary,2018-01-01,45.230000,4000000.0,8843151.0,21.809999,18340292.0,34940836.0,NaN,No
1,Afghanistan,AF,AFG,Parliamentary,2010-01-01,35.139999,4216594.0,12000000.0,29.709999,14191908.0,29120728.0,4.42,No
2,Afghanistan,AF,AFG,Parliamentary,2005-01-01,49.369999,6406615.0,12977336.0,51.709999,12389532.0,26334702.0,5.10,No
3,Afghanistan,AF,AFG,Presidential,2019-01-01,19.000000,1824401.0,9600000.0,9.630000,18938368.0,35780456.0,NaN,No
4,Afghanistan,AF,AFG,Presidential,2014-06-14,38.900002,8109493.0,20845988.0,50.029999,16208255.0,31822848.0,1.68,No


In [18]:
%%time
query = """
SELECT Country, avg("VAP Turnout") as Turnout
FROM data 
WHERE "Compulsory voting" = 'No'
GROUP BY Country 
ORDER BY Turnout DESC
LIMIT 5
"""
result = (duckdb.query(query).to_df())
print(result)

                        Country     Turnout
0                       Croatia  315.038799
1  North Macedonia, Republic of  293.369565
2                       Somalia  129.466665
3                  Cook Islands  100.639999
4                      Viet Nam   96.898000
CPU times: user 15.6 ms, sys: 3.59 ms, total: 19.2 ms
Wall time: 15.6 ms


In [17]:
%%time
filtered_data = data[data["Compulsory voting"] == "No"]
filtered_data[["Country", "VAP Turnout"]].groupby("Country")["VAP Turnout"].agg("mean").sort_values(ascending=False).head()

CPU times: user 5.58 ms, sys: 2.47 ms, total: 8.05 ms
Wall time: 5.9 ms


Country
Croatia                         315.038818
North Macedonia, Republic of    293.369568
Somalia                         129.466660
Cook Islands                    100.639992
Viet Nam                         96.897995
Name: VAP Turnout, dtype: float32

In [15]:
com_voting = data[data["Compulsory voting"] == "Yes"]
non_com_voting = data[data["Compulsory voting"] == "No"]

com_voting[["Country", "Invalid votes"]].groupby("Country")["Invalid votes"].agg(["mean", "median"])

,mean,median
Country,,
Argentina,6.137873,4.200000
Australia,3.404667,2.920000
Austria,3.093750,2.100000
Belgium,6.788750,6.700000
Bolivia,7.682059,6.700000
Brazil,11.395526,8.670000
Bulgaria,1.882000,1.040000
Chad,0.380000,0.380000
Chile,4.009167,3.150000


In [16]:
non_com_voting[["Country", "Invalid votes"]].groupby("Country")["Invalid votes"].agg(["mean", "median"])

,mean,median
Country,,
Afghanistan,3.840000,4.545
Albania,2.828333,2.480
Algeria,10.982000,11.150
Andorra,4.257500,4.485
Angola,8.228333,8.300
...,...,...
Yemen,7.000000,7.000
"Yugoslavia, FR/Union of Serbia and Montenegro",3.133333,3.100
"Yugoslavia, SFR (1943-1992)",11.000000,11.000


In [20]:
data.groupby("Compulsory voting", observed=True)["Invalid votes"].agg(["mean", "median"])

,mean,median
Compulsory voting,,
No,2.907208,1.80
Yes,5.933224,3.86
